In [1]:
import numpy as np
from gym import Env, spaces
from gym.utils import seeding

# 從類別機率中抽樣一個索引
def categorical_sample(prob_n, np_random):
    """
    從一個類別分佈中抽樣(Categorical distribution)
    prob_n: 各類別的機率(總和為1)
    np_random: 隨機數生成器(由Gym封裝)
    """

    prob_n = np.asarray(prob_n)                    # 將輸入的機率列表轉換成NumPy陣列，方便後續數值運算
    csprob_n = np.cumsum(prob_n)                   # 累加機率分佈
    return (csprob_n > np_random.rand()).argmax()  # 找到隨機值落在哪個機率區段

# 自定義離散環境類別，繼承自Gym的Env類別
class DiscreteEnv(Env):
    """
    離散型環境基礎類別，包含以下成員：
    - nS: 狀態數量
    - nA: 動作數量
    - P: 狀態轉移機率表(如下格式)
          P[s][a] = [(機率, 下一狀態, 獎勵, 是否結束), ...]
    - isd: 初始狀態機率分布
    """

    def __init__(self, nS, nA, P, isd):
        self.P = P                 # 狀態轉移表
        self.isd = isd             # 初始狀態分佈
        self.lastaction = None     # 上一次的動作(for render/debug)
        self.nS = nS               # 狀態數
        self.nA = nA               # 動作數

        # 定義Gym格式的動作與狀態空間(使用離散空間)
        self.action_space = spaces.Discrete(self.nA)
        self.observation_space = spaces.Discrete(self.nS)

        # 設定隨機種子並初始化隨機物件
        self.seed()

        # 初始狀態從isd中抽樣
        self.s = categorical_sample(self.isd, self.np_random)

    def seed(self, seed = None):
        """
        設定隨機種子(使得訓練可重現)
        """
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

    def reset(self):
        """
        將環境重設為初始狀態(從初始狀態分佈isd中抽樣)
        """
        self.s = categorical_sample(self.isd, self.np_random)
        self.lastaction = None
        return int(self.s)  # 回傳初始狀態編號

    def step(self, a):
        """
        執行一個動作，根據目前狀態與動作取得下一狀態與獎勵
        a: 動作(action)
        回傳: (新狀態, 獎勵, 是否結束, 額外資訊)
        """

        # 根據狀態與動作取得轉移列表
        transitions = self.P[self.s][a]

        # 根據機率抽樣一個轉移
        i = categorical_sample([t[0] for t in transitions], self.np_random)

        # 拆解轉移結果：機率、下一狀態、獎勵、是否結束
        p, s, r, d = transitions[i]
        self.s = s                          # 更新狀態
        self.lastaction = a                 # 記錄動作
        return int(s), r, d, {"prob": p}    # 回傳新狀態、獎勵、是否結束、額外資訊

# 定義狀態轉移表P[s][a] = [(prob, next_state, reward, done), ...]
# 共有4個狀態(0~3)，2個動作(0=左、1=右)
P = {
    0: {
        0: [(1.0, 0, 0, False)],  # 從狀態0往左還是在0(無效移動)
        1: [(1.0, 1, 0, False)]   # 從狀態0往右移動到狀態1，無獎勵
    },
    1: {
        0: [(1.0, 0, 0, False)],  # 從狀態1往左回到狀態0
        1: [(1.0, 2, 0, False)]   # 從狀態1往右到狀態2
    },
    2: {
        0: [(1.0, 1, 0, False)],  # 從狀態2往左回到狀態1
        1: [(1.0, 3, 1, True)]    # 從狀態2往右到終點(狀態3)，得到獎勵1並結束
    },
    3: {
        0: [(1.0, 3, 0, True)],   # 終點不能再移動
        1: [(1.0, 3, 0, True)]
    }
}

# 初始狀態機率分布(isd)
# [1.0, 0.0, 0.0, 0.0]表示初始一定從狀態0開始
isd = [1.0, 0.0, 0.0, 0.0]

# 使用定義好的P和isd，建立環境實例
env = DiscreteEnv(nS = 4, nA = 2, P = P, isd = isd)

# 重置環境，取得初始狀態
state = env.reset()
print(f"初始狀態: {state}")

# 開始互動流程
done = False    # 是否完成遊戲(到達終點)
step_count = 0  # 步驟計數器

# 當未完成(done == False)時，持續進行互動
while not done:
    # 隨機選擇一個動作(0或1)
    action = env.action_space.sample()

    # 執行該動作，取得下一狀態、獎勵、是否結束、其他資訊
    next_state, reward, done, info = env.step(action)

    # 印出目前步驟的詳細資訊
    print("Step", step_count, ": 動作:", action, "，下一狀態:", next_state, "，獎勵:", reward, "，是否結束:", done, "| 轉移機率:", info['prob'])

    # 更新狀態與步數
    step_count += 1

初始狀態: 0
Step 0 : 動作: 0 ，下一狀態: 0 ，獎勵: 0 ，是否結束: False | 轉移機率: 1.0
Step 1 : 動作: 0 ，下一狀態: 0 ，獎勵: 0 ，是否結束: False | 轉移機率: 1.0
Step 2 : 動作: 0 ，下一狀態: 0 ，獎勵: 0 ，是否結束: False | 轉移機率: 1.0
Step 3 : 動作: 1 ，下一狀態: 1 ，獎勵: 0 ，是否結束: False | 轉移機率: 1.0
Step 4 : 動作: 1 ，下一狀態: 2 ，獎勵: 0 ，是否結束: False | 轉移機率: 1.0
Step 5 : 動作: 0 ，下一狀態: 1 ，獎勵: 0 ，是否結束: False | 轉移機率: 1.0
Step 6 : 動作: 0 ，下一狀態: 0 ，獎勵: 0 ，是否結束: False | 轉移機率: 1.0
Step 7 : 動作: 1 ，下一狀態: 1 ，獎勵: 0 ，是否結束: False | 轉移機率: 1.0
Step 8 : 動作: 1 ，下一狀態: 2 ，獎勵: 0 ，是否結束: False | 轉移機率: 1.0
Step 9 : 動作: 1 ，下一狀態: 3 ，獎勵: 1 ，是否結束: True | 轉移機率: 1.0


Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.
c:\Users\hankc\anaconda3\Lib\site-packages\gym\utils\seeding.py:41: DeprecationWarning: WARN: Function `rng.rand(*size)` is marked as deprecated and will be removed in the future. Please use `Generator.random(size)` instead.
  deprecation(
